# Create model

[See Google MediaPipe docs for more info.](https://developers.google.com/mediapipe/solutions/customization/object_detector)

This was run in Google Colab. Adjustments may be needed to run locally.

In [ ]:
!python --version
!pip install --upgrade pip
!pip install mediapipe-model-maker

In [ ]:
from google.colab import files
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import object_detector

In [ ]:
# Pull from drive (or wherever the dataset is stored)
!tar -zxvf /content/drive/MyDrive/datasets/mtg-card-images.tar.gz

In [24]:
# Remove a bunch of garbage files from the untarring process
# !ls -la data/train
# !find ./data/ -name "._*" # -exec /bin/rm {} \;

In [25]:
train_dataset_path = "data/train"
validation_dataset_path = "data/validation"

with open(os.path.join(train_dataset_path, "labels.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")

0: background
1: card


In [ ]:
###############################
# BELOW DOES NOT RUN IN COLAB #
###############################


# import matplotlib.pyplot as plt
# from matplotlib import patches, text, patheffects
# from collections import defaultdict
# import math

# def draw_outline(obj):
#   obj.set_path_effects([patheffects.Stroke(linewidth=4,  foreground='black'), patheffects.Normal()])
# def draw_box(ax, bb):
#   patch = ax.add_patch(patches.Rectangle((bb[0],bb[1]), bb[2], bb[3], fill=False, edgecolor='red', lw=2))
#   draw_outline(patch)
# def draw_text(ax, bb, txt, disp):
#   text = ax.text(bb[0],(bb[1]-disp),txt,verticalalignment='top'
#   ,color='white',fontsize=10,weight='bold')
#   draw_outline(text)
# def draw_bbox(ax, annotations_list, id_to_label, image_shape):
#   for annotation in annotations_list:
#     cat_id = annotation["category_id"]
#     bbox = annotation["bbox"]
#     draw_box(ax, bbox)
#     draw_text(ax, bbox, id_to_label[cat_id], image_shape[0] * 0.05)
# def visualize(dataset_folder, max_examples=None):
#   with open(os.path.join(dataset_folder, "labels.json"), "r") as f:
#     labels_json = json.load(f)
#   images = labels_json["images"]
#   cat_id_to_label = {item["id"]:item["name"] for item in labels_json["categories"]}
#   image_annots = defaultdict(list)
#   for annotation_obj in labels_json["annotations"]:
#     image_id = annotation_obj["image_id"]
#     image_annots[image_id].append(annotation_obj)

#   if max_examples is None:
#     max_examples = len(image_annots.items())
#   n_rows = math.ceil(max_examples / 3)
#   fig, axs = plt.subplots(n_rows, 3, figsize=(24, n_rows*8)) # 3 columns(2nd index), 8x8 for each image
#   for ind, (image_id, annotations_list) in enumerate(list(image_annots.items())[:max_examples]):
#     ax = axs[ind//3, ind%3]
#     img = plt.imread(os.path.join(dataset_folder, "images", images[image_id]["file_name"]))
#     ax.imshow(img)
#     draw_bbox(ax, annotations_list, cat_id_to_label, img.shape)
#   plt.show()

# visualize(train_dataset_path, 9)

In [26]:
train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir="/tmp/od_data/validation")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)

train_data size:  25
validation_data size:  4


In [27]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

In [28]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128)   144928    
                             , '4': (None, 16, 16, 1             
                             28),                 

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


3/3 [==============================] - 66s 5s/step - total_loss: 6.0691 - cls_loss: 5.8070 - box_loss: 0.0040 - model_loss: 6.0062 - val_total_loss: 1.6027 - val_cls_loss: 1.4734 - val_box_loss: 0.0013 - val_model_loss: 1.5399
Epoch 2/30
3/3 [==============================] - 6s 2s/step - total_loss: 1.6628 - cls_loss: 1.3578 - box_loss: 0.0048 - model_loss: 1.6000 - val_total_loss: 1.3186 - val_cls_loss: 1.1820 - val_box_loss: 0.0015 - val_model_loss: 1.2558
Epoch 3/30
3/3 [==============================] - 8s 3s/step - total_loss: 1.5011 - cls_loss: 1.2376 - box_loss: 0.0040 - model_loss: 1.4382 - val_total_loss: 1.2552 - val_cls_loss: 1.1322 - val_box_loss: 0.0012 - val_model_loss: 1.1923
Epoch 4/30
3/3 [==============================] - 5s 2s/step - total_loss: 1.3513 - cls_loss: 1.1568 - box_loss: 0.0026 - model_loss: 1.2884 - val_total_loss: 1.1920 - val_cls_loss: 1.0715 - val_box_loss: 0.0012 - val_model_loss: 1.1291
Epoch 5/30
3/3 [==============================] - 7s 2s/step -

In [29]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

1/1 [==============================] - 1s 1s/step - total_loss: 0.3404 - cls_loss: 0.2434 - box_loss: 6.8233e-04 - model_loss: 0.2775
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.733
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.859
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.832
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.733
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.529
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.771
 Average Recall     (AR) @[

In [30]:
model.export_model()
!ls exported_model
files.download('exported_model/model.tflite')

Exporting a floating point model


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


float_ckpt.data-00000-of-00001	float_ckpt.index  metadata.json  model.tflite


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>